In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch.utils.data import random_split, DataLoader, TensorDataset
%matplotlib inline

import torch.optim as optim
import time
import random

# import torchvision.models as models

In [2]:
random.seed(100)

train_ds= torchvision.datasets.CIFAR10(root="data/", train = True, transform=transforms.ToTensor(), download=True)
test_ds= torchvision.datasets.CIFAR10(root="data/", train = False, transform=transforms.ToTensor(), download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
batch_size=128
train_loader= DataLoader(train_ds, batch_size, shuffle =True)
test_leader=  DataLoader(test_ds, batch_size, shuffle = False)
# len(train_loader)

In [4]:
device=torch.device("cuda:1" if torch.cuda.is_available else "cpu")

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


num_classes=10
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
class Bottleneck(nn.Module):
    def __init__(self, inplane, planes, stride=1):
        super(Bottleneck,self).__init__()
        outplane=4*planes
        self.conv1 =   nn.Conv2d(inplane, planes,kernel_size=1,bias=False)  #same convolution
        self.batchN1 = nn.BatchNorm2d(planes)
        self.conv2 =   nn.Conv2d(planes, planes,kernel_size=3 ,stride=stride, padding=1 ,bias=False) #if stride is 1 then same convolution
        self.batchN2 =  nn.BatchNorm2d(planes)
        self.conv3 =   nn.Conv2d(planes, outplane,kernel_size=1,bias=False) #same convolution
        self.batchN3 =  nn.BatchNorm2d(outplane)
        
        self.shortcuts=nn.Sequential()  #creating a shortcut which will help in residual part
        if(outplane!=inplane or stride!=1):  #we make the shortcut so that the dimension match 
            self.shortcuts = nn.Sequential(nn.Conv2d(inplane, outplane, kernel_size=1,stride=stride,bias=False) ,nn.BatchNorm2d(outplane))
            
    def forward(self, x):
        out=F.relu(self.batchN1(self.conv1(x)))
        out=F.relu(self.batchN2(self.conv2(out)))
        out=self.batchN3(self.conv3(out))
        out=F.relu(out+self.shortcuts(x))   #most important step of residual network
        return out


In [6]:
class Resmodel(nn.Module):
    # def __init__(self, block=Bottleneck, no_bl=[3,4,6,3], output_no=10):
    def __init__(self, block, no_bl=[3,4,6,3], output_no=10):
        super(Resmodel,self).__init__()
        self.in_plane=64
        self.conv1= nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1 ,bias=False)
        self.batchN11= nn.BatchNorm2d(64)
        self.layer1= self._make_layer(block, 64, 3, 1)
        self.layer2= self._make_layer(block, 128, 4, 2)
        self.layer3= self._make_layer(block, 256, 6, 2)
        self.layer4= self._make_layer(block, 512, 3, 2)
        self.linear = nn.Linear(2048,10)    #finally we will classify in 10 class,hence we have 10 dimension
        
    def _make_layer(self, block, plane, numofblock, stride):
        layers=[]
        strides= [stride] + [1]*(numofblock-1) #first block will have the given stride rest will have stride=1
        #we have stride only for the first bottleneck(that too only one layer of bottleneck will use it)(hence the reduction of dimension due to stride will take place only once for one block of bottleneck)
        for st in strides:
            layers.append(Bottleneck(self.in_plane, plane, st))
            self.in_plane = plane*4     #we know that the last layer in bottleneck is 4 times the size of input layer, hence we multiply it here, so that the first layer of next bottleneck will match the dimension of last layer of previous bottleneck.
        return nn.Sequential(*layers)   #*layers will return values stored in the list
    
    def forward(self, x):
        #size of x=[batchsize,3,32,32]
        out=self.conv1(x)           #after this operation size=[batchsize,64,32,32]
        out=self.batchN11(out)
        out=F.relu(out)
        # out= F.relu(self.batchN11(self.conv1(x)))
        out=self.layer1(out)        #after this operation the output size must be [batchsize,256,32,32]
        out=self.layer2(out)        #after this operation the output size must be [batchsize,512,16,16]
        out=self.layer3(out)        #after this operation the output size must be [batchsize,1024,8,8]
        out=self.layer4(out)        #after this operation the output size must be [batchsize,2048,4,4]
        out=F.avg_pool2d(out,4)     #after this operation the output size must be [batchsize,2048,1,1]
        out= out.view(out.size(0),-1)    #after this operation the output size must be [batchsize,1,2048]
        out=self.linear(out)         #after this operation the output size must be [batchsize,1,10]
        return out


In [7]:
#try focal losss

class focalloss(nn.Module):
    def __init__(self):
        super(focalloss,self).__init__()

    def forward(self, inp,targ, alpha=1, gamma=2):
        # print("shape of input before sigmoid",inp.shape)  #128*10
        inp=F.sigmoid(inp)
        inp=inp.view(-1)
        # print("shape of input after sigmoid",inp.shape)  #1280
        targ=targ.view(-1)
        Biloss=F.binary_cross_entropy(inp,targ, reduction='mean')
        bi_exp=torch.exp(-Biloss)
        focalLoss=alpha*(1-bi_exp)**gamma * Biloss
        return focalLoss


In [8]:
model = Resmodel(Bottleneck)
model=model.to(device)
# criterion = nn.CrossEntropyLoss()
criterion=focalloss()
optimizer= optim.SGD(model.parameters(), lr= 0.01)


In [9]:
%%time
n_total_steps = len(train_loader)
NOepo=10
for epoch in range(NOepo):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [128, 3, 32, 32]
        images = images.to(device)
        # labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        #for focal loss:
        labels= F.one_hot(labels, num_classes=10)       
        labels= labels.type(torch.FloatTensor)
        labels = labels.to(device)
        

        loss = criterion(outputs, labels)
        # break

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 16 == 0:
            print (f'Epoch [{epoch+1}/{2}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# print('max value of i=',max_val_i)
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)




/home/harsh19423/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Epoch [1/2], Step [16/391], Loss: 0.0318
Epoch [1/2], Step [32/391], Loss: 0.0290
Epoch [1/2], Step [48/391], Loss: 0.0268
Epoch [1/2], Step [64/391], Loss: 0.0252
Epoch [1/2], Step [80/391], Loss: 0.0272
Epoch [1/2], Step [96/391], Loss: 0.0243
Epoch [1/2], Step [112/391], Loss: 0.0258
Epoch [1/2], Step [128/391], Loss: 0.0250
Epoch [1/2], Step [144/391], Loss: 0.0249
Epoch [1/2], Step [160/391], Loss: 0.0244
Epoch [1/2], Step [176/391], Loss: 0.0235
Epoch [1/2], Step [192/391], Loss: 0.0245
Epoch [1/2], Step [208/391], Loss: 0.0241
Epoch [1/2], Step [224/391], Loss: 0.0230
Epoch [1/2], Step [240/391], Loss: 0.0239
Epoch [1/2], Step [256/391], Loss: 0.0239
Epoch [1/2], Step [272/391], Loss: 0.0237
Epoch [1/2], Step [288/391], Loss: 0.0232
Epoch 

In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_leader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        # for i in range(batch_size):
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 33.63 %
Accuracy of plane: 35.7 %
Accuracy of car: 52.2 %
Accuracy of bird: 6.8 %
Accuracy of cat: 14.5 %
Accuracy of deer: 38.0 %
Accuracy of dog: 32.3 %
Accuracy of frog: 34.5 %
Accuracy of horse: 31.1 %
Accuracy of ship: 50.6 %
Accuracy of truck: 40.6 %
